In [ ]:
!pip install bert-score


In [ ]:
import pandas as pd
from bert_score import score

In [1]:
df_rule = pd.read_csv("rule_based_outputs.csv")
df_pipeline = pd.read_csv("pipeline_outputs.csv")

df = pd.merge(df_rule, df_pipeline, on="Text_ID", how="outer")

gold_standards = {
    1: "Today we are celebrating the Dragon Boat Festival. In Chinese culture, this festival is a time to wish for safety, health, and good fortune in our lives. I also send these wishes to you, hoping you can share in the joy of the occasion.Thank you for passing our message to the doctor in connection with his upcoming contract review. I already received confirmation of this from the professor a few days ago, and I am truly grateful for his continued support, especially regarding our Springer proceedings publication.",
    2: "During our final discussion, I mentioned the new submission — the one we had been waiting for since last autumn. However, the updates were somewhat confusing, as they did not seem to include the full feedback from the reviewer or perhaps the editor.Still, I believe the team — despite some delays and less communication in recent days — truly gave their best effort for both the paper and our collaboration. We should all be grateful for the acceptance and for everyone’s hard work, especially since the paper finally appeared on SpringerLink last week.Could you also kindly remind me if the doctor still plans to edit the acknowledgments section before sending the final version? I didn’t see that part completed yet — or perhaps I overlooked it; if so, my apologies.In any case, let’s make sure everything is finalized, stay safe, and celebrate this outcome together with strong coffee and new goals for the future."
}

results = []

for i, row in df.iterrows():
    text_id = row["Text_ID"]
    gold = gold_standards.get(text_id, "")
    if gold.strip() == "":
        continue
    
    comparisons = {
        "Rule_Based": row.get("Rule_Based_Rewrite", ""),
        "First_Model": row.get("First_Model_Grammar", ""),
        "Second_Model": row.get("Second_Model_Summary", ""),
        "Third_Model": row.get("Third_Model_Grammar", "")
    }
    
    for key, candidate in comparisons.items():
        if candidate.strip() == "":
            results.append({
                "Text_ID": text_id,
                "Comparison": f"{key}_vs_Gold",
                "Precision": None,
                "Recall": None,
                "F1": None
            })
            continue
        
        P, R, F1 = score([candidate], [gold], lang="en", verbose=False)
        results.append({
            "Text_ID": text_id,
            "Comparison": f"{key}_vs_Gold",
            "Precision": P.mean().item(),
            "Recall": R.mean().item(),
            "F1": F1.mean().item()
        })

bertscore_vs_gold_df = pd.DataFrame(results)

bertscore_vs_gold_df


E:\anaconda3\envs\nlp2025\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Some weights of RobertaModel were not initialized from the model checkpoint at roberta-large and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this mod

,Text_ID,Comparison,Precision,Recall,F1
0,1,Rule_Based_vs_Gold,0.893862,0.893569,0.893716
1,1,First_Model_vs_Gold,0.889493,0.899390,0.894414
2,1,Second_Model_vs_Gold,0.859295,0.867231,0.863245
3,1,Third_Model_vs_Gold,0.894076,0.902209,0.898124
4,2,Rule_Based_vs_Gold,0.909965,0.911809,0.910886
5,2,First_Model_vs_Gold,0.929023,0.921276,0.925133
6,2,Second_Model_vs_Gold,0.885261,0.839643,0.861849
7,2,Third_Model_vs_Gold,0.929374,0.922204,0.925775
